# Генерация помесячных фич на полных датасетах

In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
# предопределим типы колонок для экономии памяти
dtypes = {
    'ID': np.int32,
    'CLIENT': np.int32,
    'DATA_TRANS': str,
    'AZS_NUMBER': np.int16,
    'REGION_AZS': np.int16,
    'VID_NP': np.int16,
    'COL_LITR':  np.float32,
    'CENA_CLIENT': np.float32
    }

In [3]:
def date(s):
    return s[3:10]

In [4]:
%%time
train = pd.read_csv('datasets/gpn_b2b/train_data.csv', dtype = dtypes)
train['date'] = train['DATA_TRANS'].apply(date)
train['client_date'] = train['CLIENT'].astype(str) + '_' + train['date'].astype(str)

CPU times: user 1min 50s, sys: 6.24 s, total: 1min 57s
Wall time: 1min 57s


In [5]:
train.drop(['ID','DATA_TRANS','VID_NP','AZS_NUMBER','REGION_AZS'], axis=1, inplace=True)
gc.collect()

21

In [6]:
train.head()

,CLIENT,COL_LITR,CENA_CLIENT,date,client_date
0,46219,15.000000,35.700001,11.2016,46219_11.2016
1,33792,30.000000,31.430000,11.2016,33792_11.2016
2,16215,220.000000,32.930000,11.2016,16215_11.2016
3,33792,15.000000,31.160000,11.2016,33792_11.2016
4,17075,133.669998,34.720001,11.2016,17075_11.2016


In [7]:
%%time
test_full = pd.read_csv('datasets/gpn_b2b/full_test_data.csv', dtype = dtypes)

CPU times: user 9.66 s, sys: 464 ms, total: 10.1 s
Wall time: 10.1 s


In [8]:
%%time

test_full['date'] = test_full['DATA_TRANS'].apply(date)
test_full['client_date'] = test_full['CLIENT'].astype(str) + '_' + test_full['date'].astype(str)

test_full['date'] = test_full['DATA_TRANS'].apply(date)
test_full['client_date'] = test_full['CLIENT'].astype(str) + '_' + test_full['date'].astype(str)

test_full.drop(['ID','DATA_TRANS','VID_NP','AZS_NUMBER','REGION_AZS'], axis=1, inplace=True)
gc.collect()

CPU times: user 41.1 s, sys: 2.7 s, total: 43.8 s
Wall time: 43.8 s


In [9]:
%%time

dt=train.append(test_full)

CPU times: user 2.55 s, sys: 1.28 s, total: 3.84 s
Wall time: 3.83 s


In [10]:
%%time
# количество транзакций клиента


dt = dt.merge(dt.groupby('client_date')['CENA_CLIENT'].count().reset_index(name = 'trans_count'), how = 'left')
dt['trans_count'] = dt['trans_count'].astype(np.int32)

CPU times: user 25.2 s, sys: 5.98 s, total: 31.1 s
Wall time: 32 s


In [11]:
%%time 
# медианный литраж по клиенту за всё время - т.к. в трейне литража нет

dt = dt.merge(dt.groupby('CLIENT')['COL_LITR'].mean().reset_index(name = 'ltr_mean'), how = 'left')
dt['ltr_mean'] = dt['ltr_mean']

CPU times: user 12.8 s, sys: 4.9 s, total: 17.7 s
Wall time: 17.9 s


In [13]:
dt.sample(10)

,CENA_CLIENT,CLIENT,COL_LITR,client_date,date,trans_count,ltr_mean,cena_mean
31532826,36.570000,29853,30.000000,29853_12.2017,12.2017,9731,82.189316,36.648407
40482663,37.490002,23774,NaN,23774_03.2018,03.2018,1052,36.710041,38.795143
33746259,39.939999,34661,NaN,34661_01.2018,01.2018,327,77.211266,39.316727
22101066,32.490002,30834,20.000000,30834_09.2017,09.2017,763,25.993404,32.554195
12279881,34.220001,29931,150.000000,29931_05.2017,05.2017,313,129.415405,33.328148
1550610,32.090000,48596,50.000000,48596_12.2016,12.2016,2388,59.088768,33.327644
23750631,37.049999,42404,59.049999,42404_10.2017,10.2017,163,36.918709,37.151962
1771083,32.919998,48539,250.000000,48539_12.2016,12.2016,1622,304.112732,35.134098
1691250,29.889999,49036,100.000000,49036_12.2016,12.2016,216711,64.986610,29.841076
28740,28.260000,620,40.000000,620_11.2016,11.2016,6116,37.219879,30.766994


In [15]:
%%time

dt[['client_date','trans_count','ltr_mean']].groupby('client_date').mean().to_csv('datasets/gpn_b2b/features_monthly.csv')

CPU times: user 11.5 s, sys: 1.88 s, total: 13.4 s
Wall time: 13.4 s


In [ ]:
# добавить колонку кол-во транзакций * медианный литраж, по ней добавить категорию клиента в месяце